# 自分で育てた馬を有馬記念に
**説明変数**
* グラウンド
* 馬場の状態
* 距離
* 天候
* どっち周りか
* 騎手の名前

**目的変数**
* タイム

**やることの想定**
* まずはスクレイピングで今回出馬した馬の上記の変数を取り出す。(今回はめんどくさいので1,2,3位の馬の情報のみ抽出することにする)
* その後機械学習によってそれぞれのタイムを機械学習によって推測させる。

In [1]:
# スクレイピング
from bs4 import BeautifulSoup
# HTTP操作用
import requests 
# サーバーに負担をかけないようにする
import time
# csvを作るために使うよ
import csv
# 考察に使うやつ
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pip install lxml


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# アクセスしたいWebサイトのURL
url = 'https://db.netkeiba.com/horse/result/2019105283/'

# Webサーバーにリクエストを出す
r = requests.get(url)

# print(r.text)

# このページでのエンコーディングを取得する
encoding=r.encoding
# パーサーがこれまでと同じだとうまくいかなかったためLXMLで行う
soup = BeautifulSoup(r.content, 'lxml')
# 一度見てみる
print(soup)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html id="html" lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="text/javascript" http-equiv="content-script-type"/>
<meta content="text/css" http-equiv="content-style-type"/>
<link href="https://cdn.netkeiba.com/img.db/common/css/reset.css?20160421" media="all" rel="stylesheet" type="text/css"/>
<link href="https://cdn.netkeiba.com/img.db/common/css/common.css?20210819" media="all" rel="stylesheet" type="text/css"/>
<link href="https://cdn.netkeiba.com/img.db/common/css/db_detail.css?20180621" media="all" rel="stylesheet" type="text/css"/>
<link href="https://cdn.netkeiba.com/img.db/common/css/horse_detail.css?20220606" media="all" rel="stylesheet" type="text/css"/>
<link href="https://cdn.netkeiba.c

In [6]:
# 表の特定の列を抽出
table = soup.find('table'  ,class_="db_h_race_results nk_tb_common")
# print(table)
# 列のインデックス（0から始まる）を指定
weather_column_index = 2 # 「天気」の列を抽出
name_race_column_index = 4  # 「レースの名前」列を抽出
no_column_index = 11 #「着順」列を抽出
load_weight_column_index = 13 #「負荷重量」列を抽出
long_column_index = 14 #「土壌,距離」列を抽出
condition_column_index = 15  #「状態」列を抽出
time_column_index = 17 #「タイム」列を抽出
weather_list = []
race_name_list = []
no_list = []
load_weight_list = []
long_list = []
condition_list = []
time_list = []

HEADER = ["race_name","weather","no","load_weight","long","condition","time"]
with open("horse_racing1.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(HEADER)
    # 表内の各行に対して処理
    for row in table.find_all('tr')[1:]:  # 最初の行はヘッダーなので無視
        all_list = []
        # 行内での対応する列のデータを抽出
        all_ellements = row.find_all('td')
        if all_ellements:
            weather = all_ellements[weather_column_index].text
            weather_list.append(weather)

            race_name = all_ellements[name_race_column_index].text
            race_name_list.append(race_name)

            no = all_ellements[no_column_index].text
            no_list.append(f"no")

            load_weight = all_ellements[load_weight_column_index].text
            load_weight_list.append(load_weight)

            long = all_ellements[long_column_index].text
            long_list.append(long)

            condition = all_ellements[condition_column_index].text
            condition_list.append(condition)

            time = all_ellements[time_column_index].text
            time_list.append(time)

            all_list.append(race_name)
            all_list.append(weather)
            all_list.append(no)
            all_list.append(load_weight)
            all_list.append(long)
            all_list.append(condition)
            all_list.append(time)
            print(all_list)
            
            writer.writerow(all_list)

['有馬記念(G1)', '晴', '1', '58', '芝2500', '良', '2:30.9']
['ジャパンC(G1)', '曇', '4', '58', '芝2400', '良', '2:22.7']
['天皇賞(秋)(G1)', '晴', '7', '58', '芝2000', '良', '1:56.6']
['ドバイターフ(G1)', '晴', '取', '57', '芝1800', '良', '\xa0']
['京都記念(G2)', '晴', '1', '58', '芝2200', '良', '2:10.9']
['凱旋門賞(G1)', ' ', '19', '56.5', '芝2400', '重', '\xa0']
['ニエル賞(G2)', ' ', '4', '58', '芝2400', '重', '\xa0']
['東京優駿(G1)', '晴', '1', '57', '芝2400', '良', '2:21.9']
['皐月賞(G1)', '曇', '3', '57', '芝2000', '良', '2:00.0']
['報知弥生ディープ記念(G2)', '曇', '2', '56', '芝2000', '良', '2:00.5']
['朝日フューチュリティ(G1)', '晴', '1', '55', '芝1600', '良', '1:33.5']
['アイビーS(L)', '晴', '1', '55', '芝1800', '良', '1:49.3']
['2歳新馬', '曇', '1', '54', '芝1800', '良', '1:50.2']


In [7]:
# アクセスしたいWebサイトのURL
url = 'https://db.netkeiba.com/horse/result/2019104740/'

# Webサーバーにリクエストを出す
r = requests.get(url)

# print(r.text)

# このページでのエンコーディングを取得する
encoding=r.encoding
# パーサーがこれまでと同じだとうまくいかなかったためLXMLで行う
soup = BeautifulSoup(r.content, 'lxml')
# 一度見てみる
print(soup)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html id="html" lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="text/javascript" http-equiv="content-script-type"/>
<meta content="text/css" http-equiv="content-style-type"/>
<link href="https://cdn.netkeiba.com/img.db/common/css/reset.css?20160421" media="all" rel="stylesheet" type="text/css"/>
<link href="https://cdn.netkeiba.com/img.db/common/css/common.css?20210819" media="all" rel="stylesheet" type="text/css"/>
<link href="https://cdn.netkeiba.com/img.db/common/css/db_detail.css?20180621" media="all" rel="stylesheet" type="text/css"/>
<link href="https://cdn.netkeiba.com/img.db/common/css/horse_detail.css?20220606" media="all" rel="stylesheet" type="text/css"/>
<link href="https://cdn.netkeiba.c

In [8]:
# 表の特定の列を抽出
table = soup.find('table'  ,class_="db_h_race_results nk_tb_common")
# print(table)
# 列のインデックス（0から始まる）を指定
weather_column_index = 2 # 「天気」の列を抽出
name_race_column_index = 4  # 「レースの名前」列を抽出
no_column_index = 11 #「着順」列を抽出
load_weight_column_index = 13 #「負荷重量」列を抽出
long_column_index = 14 #「土壌,距離」列を抽出
condition_column_index = 15  #「状態」列を抽出
time_column_index = 17 #「タイム」列を抽出
weather_list = []
race_name_list = []
no_list = []
load_weight_list = []
long_list = []
condition_list = []
time_list = []

HEADER = ["race_name","weather","no","load_weight","long","condition","time"]
with open("horse_racing2.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(HEADER)
    # 表内の各行に対して処理
    for row in table.find_all('tr')[1:]:  # 最初の行はヘッダーなので無視
        all_list = []
        # 行内での対応する列のデータを抽出
        all_ellements = row.find_all('td')
        if all_ellements:
            weather = all_ellements[weather_column_index].text
            weather_list.append(weather)

            race_name = all_ellements[name_race_column_index].text
            race_name_list.append(race_name)

            no = all_ellements[no_column_index].text
            no_list.append(no)

            load_weight = all_ellements[load_weight_column_index].text
            load_weight_list.append(load_weight)

            long = all_ellements[long_column_index].text
            long_list.append(long)

            condition = all_ellements[condition_column_index].text
            condition_list.append(condition)

            time = all_ellements[time_column_index].text
            time_list.append(time)

            all_list.append(race_name)
            all_list.append(weather)
            all_list.append(no)
            all_list.append(load_weight)
            all_list.append(long)
            all_list.append(condition)
            all_list.append(time)
            print(all_list)
            
            writer.writerow(all_list)

['有馬記念(G1)', '晴', '2', '56', '芝2500', '良', '2:31.0']
['ジャパンC(G1)', '曇', '3', '56', '芝2400', '良', '2:22.6']
['ヴィクトリアマイル(G1)', '小雨', '3', '56', '芝1600', '良', '1:32.3']
['大阪杯(G1)', '晴', '2', '56', '芝2000', '良', '1:57.4']
['秋華賞(G1)', '晴', '3', '55', '芝2000', '良', '1:58.7']
['優駿牝馬(G1)', '晴', '1', '55', '芝2400', '良', '2:23.9']
['桜花賞(G1)', '晴', '1', '55', '芝1600', '良', '1:32.9']
['デイリー杯クイーンC(G3)', '晴', '2', '54', '芝1600', '良', '1:34.2']
['フェアリーS(G3)', '曇', '2', '54', '芝1600', '良', '1:35.3']
['赤松賞(1勝クラス)', '曇', '3', '54', '芝1600', '良', '1:34.3']
['2歳未勝利', '曇', '1', '54', '芝1800', '良', '1:47.3']
['2歳新馬', '雨', '2', '54', '芝1800', '良', '1:51.3']


In [9]:
# アクセスしたいWebサイトのURL
url = 'https://db.netkeiba.com/horse/result/2018103559/'

# Webサーバーにリクエストを出す
r = requests.get(url)

# print(r.text)

# このページでのエンコーディングを取得する
encoding=r.encoding
# パーサーがこれまでと同じだとうまくいかなかったためLXMLで行う
soup = BeautifulSoup(r.content, 'lxml')
# 一度見てみる
print(soup)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html id="html" lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="text/javascript" http-equiv="content-script-type"/>
<meta content="text/css" http-equiv="content-style-type"/>
<link href="https://cdn.netkeiba.com/img.db/common/css/reset.css?20160421" media="all" rel="stylesheet" type="text/css"/>
<link href="https://cdn.netkeiba.com/img.db/common/css/common.css?20210819" media="all" rel="stylesheet" type="text/css"/>
<link href="https://cdn.netkeiba.com/img.db/common/css/db_detail.css?20180621" media="all" rel="stylesheet" type="text/css"/>
<link href="https://cdn.netkeiba.com/img.db/common/css/horse_detail.css?20220606" media="all" rel="stylesheet" type="text/css"/>
<link href="https://cdn.netkeiba.c

In [12]:
    # 表の特定の列を抽出
    table = soup.find('table'  ,class_="db_h_race_results nk_tb_common")
    # print(table)
    # 列のインデックス（0から始まる）を指定
    weather_column_index = 2 # 「天気」の列を抽出
    name_race_column_index = 4  # 「レースの名前」列を抽出
    no_column_index = 11 #「着順」列を抽出
    load_weight_column_index = 13 #「負荷重量」列を抽出
    long_column_index = 14 #「土壌,距離」列を抽出
    condition_column_index = 15  #「状態」列を抽出
    time_column_index = 17 #「タイム」列を抽出
    weather_list = []
    race_name_list = []
    no_list = []
    load_weight_list = []
    long_list = []
    condition_list = []
    time_list = []

    HEADER = ["race_name","weather","no","load_weight","long","condition","time"]
    with open("horse_racing3.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(HEADER)
        # 表内の各行に対して処理
        for row in table.find_all('tr')[1:]:  # 最初の行はヘッダーなので無視
            all_list = []
            # 行内での対応する列のデータを抽出
            all_ellements = row.find_all('td')
            if all_ellements:
                weather = all_ellements[weather_column_index].text
                weather_list.append(weather)

                race_name = all_ellements[name_race_column_index].text
                race_name_list.append(race_name)

                no = all_ellements[no_column_index].text
                no_list.append(no)

                load_weight = all_ellements[load_weight_column_index].text
                load_weight_list.append(load_weight)

                long = all_ellements[long_column_index].text
                long_list.append(long)

                condition = all_ellements[condition_column_index].text
                condition_list.append(condition)

                time = all_ellements[time_column_index].text
                time_list.append(time)

                all_list.append(race_name)
                all_list.append(weather)
                all_list.append(no)
                all_list.append(load_weight)
                all_list.append(long)
                all_list.append(condition)
                all_list.append(time)
                print(all_list)
                
                writer.writerow(all_list)

['有馬記念(G1)', '晴', '3', '58', '芝2500', '良', '2:31.2']
['ジャパンC(G1)', '曇', '5', '58', '芝2400', '良', '2:23.1']
['産経賞オールカマー(G2)', '晴', '2', '58', '芝2200', '良', '2:12.2']
['天皇賞(春)(G1)', '曇', '中', '58', '芝3200', '稍', '\xa0']
['日経賞(G2)', '曇', '1', '59', '芝2500', '不', '2:36.8']
['有馬記念(G1)', '晴', '9', '57', '芝2500', '良', '2:34.1']
['凱旋門賞(G1)', ' ', '11', '59.5', '芝2400', '重', '\xa0']
['宝塚記念(G1)', '晴', '1', '58', '芝2200', '良', '2:09.7']
['天皇賞(春)(G1)', '晴', '1', '58', '芝3200', '稍', '3:16.2']
['日経賞(G2)', '曇', '1', '57', '芝2500', '稍', '2:35.4']
['有馬記念(G1)', '晴', '5', '55', '芝2500', '良', '2:32.5']
['菊花賞(G1)', '曇', '1', '57', '芝3000', '良', '3:04.6']
['朝日セントライト記念(G2)', '晴', '13', '56', '芝2200', '良', '2:13.6']
['東京優駿(G1)', '晴', '6', '57', '芝2400', '良', '2:23.1']
['皐月賞(G1)', '晴', '2', '57', '芝2000', '稍', '2:01.1']
['報知弥生ディープ記念(G2)', '曇', '1', '56', '芝2000', '良', '2:02.0']
['ホープフルS(G1)', '晴', '4', '55', '芝2000', '良', '2:03.3']
['東京スポーツ杯2歳S(G3)', '曇', '2', '55', '芝1800', '良', '1:47.7']
['2歳新馬', '曇', '1', '

In [13]:
# sqliteをimportする
import sqlite3

In [14]:
# どこにDBファイルを保存するかの確認。
!pwd

/Users/a11111/DSprograming/final_assignment


In [ ]:
# DBファイルを保存するためのファイルパス

# Google Colab
path = './'


# DBファイル名
db_name = 'horse_data.sqlite'

# DBに接続する（指定したDBファイル存在しない場合は，新規に作成される）
con = sqlite3.connect(path + db_name)

# DBへの接続を閉じる
con.close()